In [104]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.metrics import log_loss

from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import ExtraTreesClassifier

from sklearn import svm


In [175]:
df = pd.read_csv("../MachinehackGlassQualityPrediction/Glass_Quality_Participants_Data/Train.csv")
#tf = pd.read_csv("../MachinehackGlassQualityPrediction/Glass_Quality_Participants_Data/Test.csv")

#sf = pd.read_excel("../MachinehackGlassQualityPrediction/Glass_Quality_Participants_Data/Sample_Submission.xlsx")


In [176]:
# df['xmax'] = np.sqrt(df['xmax'])
# df['ymax'] = np.sqrt(df['ymax'])
# df['xmin'] = np.sqrt(df['xmin'])
# df['ymin'] = np.sqrt(df['ymin'])

# df['max_luminosity'] = np.log(df['max_luminosity'])
# df['pixel_area'] = np.sqrt(df['pixel_area'])
# df['log_area'] = np.sqrt(df['log_area'])

# # tf['xmax'] = np.sqrt(tf['xmax'])
# # tf['ymax'] = np.sqrt(tf['ymax'])
# # tf['xmin'] = np.sqrt(tf['xmin'])
# # tf['ymin'] = np.sqrt(tf['ymin'])

# # tf['max_luminosity'] = np.log(tf['max_luminosity'])
# # tf['pixel_area'] = np.sqrt(tf['pixel_area'])
# # tf['log_area'] = np.sqrt(tf['log_area'])



In [177]:
#df =df.drop('pixel_area',axis=1)

# tf =tf.drop('log_area',axis=1)

In [178]:
# adding  interaction new features 
import itertools
interactions = pd.DataFrame(index=df.index)
interactions

""
0
1
2
3
4
...
1353
1354
1355
1356


In [179]:
features =['max_luminosity','thickness','pixel_area']
for col1 ,col2 in  itertools.combinations(features,2):
    
    newcolname = col1 + "_" + col2 
    new_values = df[col1].map(str) + "_" + df[col2].map(str)
    interactions[newcolname] = new_values


In [180]:
# interactions['max_luminosity_thickness']

In [181]:
#df =df.join(interactions)

In [182]:
# interactionstest = pd.DataFrame(index=tf.index)
# interactionstest

In [183]:
# features =['max_luminosity','thickness','pixel_area']
# for col1 ,col2 in  itertools.combinations(features,2):
    
#     newcolname = col1 + "_" + col2 
#     new_values = tf[col1].map(str) + "_" + tf[col2].map(str)
#     interactionstest[newcolname] = new_values


In [184]:
# tf =tf.join(interactionstest)

In [185]:
# tf
# df['class'] = df['class'].replace(2,0)

In [186]:
df.head()

,grade_A_Component_1,grade_A_Component_2,max_luminosity,thickness,xmin,xmax,ymin,ymax,pixel_area,log_area,x_component_1,x_component_2,x_component_3,x_component_4,x_component_5,class
0,0,1,150,36,144,172,947225,947332,439,439.099273,0,0,1,0,0,1
1,1,0,134,55,1144,1152,2379058,2379624,329,329.205616,1,0,0,0,0,1
2,1,0,135,65,950,974,1038442,1036754,300,300.120598,0,0,0,0,0,2
3,0,1,123,35,41,220,1705580,1705604,6803,6803.778622,0,0,1,0,0,1
4,1,0,138,69,462,466,1088124,1086579,251,251.401943,0,0,0,0,0,2


In [306]:
y = df['class']

# To keep things simple, we'll use only numerical predictors
X = df.drop(['class'], axis=1)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.70, test_size=0.30,random_state=0)

In [307]:
# enc = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='kmeans')
# enc.fit(np.array(X_train['max_luminosity']).reshape(-1,1))
# X_train['max_luminosity_binned'] = enc.transform(np.array(X_train['max_luminosity']).reshape(-1,1))
# X_valid['max_luminosity_binned'] = enc.transform(np.array(X_valid['max_luminosity']).reshape(-1,1))


In [308]:
enc = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='kmeans')
enc.fit(np.array(X_train['thickness']).reshape(-1,1))
X_train['thickness_binned'] = enc.transform(np.array(X_train['thickness']).reshape(-1,1))
X_valid['thickness_binned'] = enc.transform(np.array(X_valid['thickness']).reshape(-1,1))


/home/ryan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ryan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [309]:
X_train.head()

,grade_A_Component_1,grade_A_Component_2,max_luminosity,thickness,xmin,xmax,ymin,ymax,pixel_area,log_area,x_component_1,x_component_2,x_component_3,x_component_4,x_component_5,thickness_binned
998,0,1,128,35,41,218,2897890,2897153,6783,6783.901312,0,0,1,0,0,0.0
862,0,1,148,45,491,501,4186854,4187826,70,70.663894,0,0,0,0,0,0.0
1019,0,1,145,39,790,1316,3544945,3544655,8447,8447.198243,0,0,0,0,0,0.0
16,1,0,125,71,53,61,2644740,2645020,196,196.072339,0,1,0,0,0,1.0
1011,0,1,134,54,4,777,18824,18544,14416,14416.043888,0,0,1,0,0,0.0


In [310]:

def targetencoding(train,test,y_train):
    import category_encoders as ce
    # Create the encoder itself
    cat_features = ['max_luminosity_thickness','max_luminosity_pixel_area','thickness_pixel_area']
    print(f'targest emcoding for features {cat_features}')
    target_enc = ce.TargetEncoder(cols=cat_features)

    

    # Fit the encoder using the categorical features and target
    target_enc.fit(train[cat_features], y_train)
    

    # Transform the features, rename the columns with _target suffix, and join to dataframe
    traintrgtenc = train.join(target_enc.transform(train[cat_features]).add_suffix('_target'))
    testtrgtenc = test.join(target_enc.transform(test[cat_features]).add_suffix('_target'))

    traintrgtenc = traintrgtenc.drop(cat_features, axis =1)
    testtrgtenc = testtrgtenc.drop(cat_features, axis =1)

    
    print(traintrgtenc.shape,testtrgtenc.shape)
    
    return traintrgtenc,testtrgtenc

In [311]:
#X_traint,X_validt = targetencoding(X_train,X_valid,y_train)

In [312]:
X_train.head()

,grade_A_Component_1,grade_A_Component_2,max_luminosity,thickness,xmin,xmax,ymin,ymax,pixel_area,log_area,x_component_1,x_component_2,x_component_3,x_component_4,x_component_5,thickness_binned
998,0,1,128,35,41,218,2897890,2897153,6783,6783.901312,0,0,1,0,0,0.0
862,0,1,148,45,491,501,4186854,4187826,70,70.663894,0,0,0,0,0,0.0
1019,0,1,145,39,790,1316,3544945,3544655,8447,8447.198243,0,0,0,0,0,0.0
16,1,0,125,71,53,61,2644740,2645020,196,196.072339,0,1,0,0,0,1.0
1011,0,1,134,54,4,777,18824,18544,14416,14416.043888,0,0,1,0,0,0.0


In [384]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
    
# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    
    
    clf1 = ExtraTreesClassifier(n_estimators=100,criterion= 'gini',warm_start=True,
                                oob_score=False,bootstrap=False,max_features='auto',
                                random_state=2)

    clf1.fit(X_train,y_train)
    
    
    preds = clf1.predict(X_valid)
    predsforlog = clf1.predict_proba(X_valid)
    
    
    target_names = ['class_0','class_1']
    print(classification_report(y_valid, preds, target_names=target_names,labels= [1,2]))
    
    print(confusion_matrix(y_valid,preds))
    return log_loss(y_valid,predsforlog) 

In [385]:
X_train.dtypes

grade_A_Component_1      int64
grade_A_Component_2      int64
max_luminosity           int64
thickness                int64
xmin                     int64
xmax                     int64
ymin                     int64
ymax                     int64
pixel_area               int64
log_area               float64
x_component_1            int64
x_component_2            int64
x_component_3            int64
x_component_4            int64
x_component_5            int64
thickness_binned       float64
dtype: object

In [386]:
n_folds=5
def logloss(model):
    kf = KFold(n_folds, shuffle=True, random_state=2).get_n_splits(X_train.values)
    logloss= (-cross_val_score(model, X_train.values, y_train, scoring="neg_log_loss", cv = kf))
    return(logloss)

In [387]:
 score_dataset(X_train,X_valid,y_train,y_valid)

              precision    recall  f1-score   support

     class_0       0.91      0.90      0.90       258
     class_1       0.82      0.84      0.83       150

    accuracy                           0.88       408
   macro avg       0.86      0.87      0.87       408
weighted avg       0.88      0.88      0.88       408

[[231  27]
 [ 24 126]]


0.2623923587967047

In [388]:
for i in [400] :

    l = logloss(
            ExtraTreesClassifier(n_estimators=i,criterion= 'entropy',random_state=2,bootstrap=False,oob_score=False)
        
    )
    print(i)

    print("(avg {0} ,std{1})".format(l.mean(),l.std()))


400
(avg 0.2853913980070917 ,std0.011064157992007152)


In [369]:
ExtraTreesClassifier(random_state=2)

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100,
                     n_jobs=None, oob_score=False, random_state=2, verbose=0,
                     warm_start=False)